<a href="https://colab.research.google.com/github/schachtau/WIDS_Model/blob/main/WiDS2021_Final_Model_for_Diabetes_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

widsdatathon2021_path = kagglehub.competition_download('widsdatathon2021')

print('Data source import complete.')


The objective of this notebook is to create models to predict diabetes in ICU patients. A detailed explanation can be accessed [here](https://towardsdatascience.com/wids-datathon-2021-my-first-kaggle-datathon-806a99cbfb19?sk=b4883e44c6985380c213cf313d690405).


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy import mean
from numpy import std

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from catboost import Pool, cv, CatBoostClassifier

from sklearn.feature_selection import RFECV

from sklearn.metrics import classification_report, auc, roc_auc_score, accuracy_score, recall_score, precision_score, f1_score, plot_confusion_matrix, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, VotingClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.svm import SVC

from xgboost import XGBClassifier
import lightgbm as lgb
import xgboost as xgb


from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report,confusion_matrix, roc_curve
#parameter tuning
from sklearn.model_selection import GridSearchCV
from matplotlib.legend_handler import HandlerLine2D


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#loading dataset
train_dataset = pd.read_csv('/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv')
test_dataset = pd.read_csv('/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv')
data_dict = pd.read_csv('/kaggle/input/widsdatathon2021/DataDictionaryWiDS2021.csv')
print(train_dataset.shape)
print(test_dataset.shape)

In [ ]:
train_dataset['train_data']=1
test_dataset['train_data']=0
test_dataset['diabetes_mellitus']=np.NaN
all_data=pd.concat([train_dataset, test_dataset])


In [ ]:
#analysing apache_cols and vital_cols as they have lot of duplicates
apache_cols = all_data.columns[all_data.columns.str.contains('apache')]
apache_cols = [c.split('_apache')[0] for c in apache_cols]

vital_cols = all_data.columns[all_data.columns.str.startswith('d1') & all_data.columns.str.contains('_max')]
vital_cols = [(c.split('d1_')[1]).split('_max')[0] for c in vital_cols]

common_cols = [c for c in apache_cols if c in vital_cols]

for c in common_cols:
    var1 = f"d1_{c}_max"
    var2 = f"{c}_apache"
    notna_condition = all_data[var1].notna() & all_data[var2].notna()

    print(f"{c} has {np.round((all_data[notna_condition][var2]==(all_data[notna_condition][var1])).sum()/len(all_data[notna_condition])*100,2)}% duplicates")




In [ ]:
#filling missing values for d1 cols using corresponding apache cols
for c in common_cols:
    if c not in ['resprate', 'temp']:
        # Fill empty d1_..._max column from available ..._apache column
        all_data[f"d1_{c}_max"] = np.where((all_data[f"d1_{c}_max"].isna()
                                            & all_data[f"{c}_apache"].notna()),
                                           all_data[f"{c}_apache"],
                                           all_data[f"d1_{c}_max"])

        # Drop ..._apache column
        all_data.drop(f"{c}_apache", axis=1, inplace=True)


# Fill empty d1_heartrate_max column from available heart_rate_apache column
all_data["d1_heartrate_max"] = np.where((all_data["d1_heartrate_max"].isna()
                                    & all_data["heart_rate_apache"].notna()),
                                   all_data["heart_rate_apache"],
                                   all_data["d1_heartrate_max"])

# Drop ..._apache column
all_data.drop("heart_rate_apache", axis=1, inplace=True)


In [ ]:
#interchanging f1_max and f1_min values where f1_max<f1_min
for col in vital_cols:
    min_col = f"d1_{col}_min"
    max_col = f"d1_{col}_max"
    all_data.loc[all_data[min_col] > all_data[max_col],[min_col, max_col]] = all_data.loc[all_data[min_col] > all_data[max_col], [max_col, min_col]].values


In [ ]:
#identifying columns having same values and dropping them
cols_to_drop = []
for i, col_1 in enumerate(all_data.columns):
    for col_2 in all_data.columns[(i+1):]:
        if all_data[col_1].equals(all_data[col_2]):
            print(f"{col_1} and {col_2} are identical.")
            cols_to_drop.append(col_2)

all_data.drop(cols_to_drop, axis=1, inplace = True)

The oxigenation index or Horowitz index is the ratio of partial pressure of oxygen in blood (PaO2) and the fraction of oxygen in the inhaled air (FiO2). Therefore, we can fill the missing values of d1_pao2fio2ratio_max with pao2_apache/fio2_apache.

In [ ]:
#filling missing values for 'd1_pao2fio2ratio_max'
all_data["d1_pao2fio2ratio_max"] = np.where((all_data["pao2_apache"].notna()
                                             & all_data["fio2_apache"].notna()
                                             & all_data["d1_pao2fio2ratio_max"].isna() ),
                                            all_data["pao2_apache"] / all_data["fio2_apache"],
                                            all_data["d1_pao2fio2ratio_max"])

In [ ]:
#dropping h1 cols as h1 columns have lot of missing values and h1 and d1 values are similar
drop_columns = all_data.columns[all_data.columns.str.startswith('h1')]
all_data.drop(drop_columns, axis=1, inplace=True)

In [ ]:
all_data.drop(columns = ['Unnamed: 0', 'encounter_id', 'hospital_id', 'readmission_status'], inplace=True)

In [ ]:
#Dropping rows with age=0
all_data = all_data[all_data['age']>=16].reset_index(drop=True)
#filling missing values of 'ethnicity' col as 'Other/Unknown' as we have more than 5000 rows with unknown value
all_data['ethnicity'] = all_data['ethnicity'].fillna('Other/Unknown')
all_data['gender'] = all_data['gender'].fillna('M')

In [ ]:
all_data['height'] = all_data.groupby('gender')['height'].transform(lambda x: x.fillna(x.mean()))
all_data['weight'] = all_data.groupby('gender')['weight'].transform(lambda x: x.fillna(x.mean()))
all_data['bmi'] = all_data.groupby('gender')['bmi'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
all_data.loc[all_data['hospital_admit_source'] == 'Acute Care/Floor', 'hospital_admit_source'] = 'Floor'
all_data.loc[all_data['hospital_admit_source'] == 'Step-Down Unit (SDU)', 'hospital_admit_source'] = 'SDU'
all_data.loc[all_data['hospital_admit_source'] == 'ICU to SDU', 'hospital_admit_source'] = 'SDU'
all_data.loc[all_data['hospital_admit_source'] == 'Other ICU', 'hospital_admit_source'] = 'ICU'
all_data.loc[all_data['hospital_admit_source'] == 'PACU', 'hospital_admit_source'] = 'Recovery Room'
all_data.loc[all_data['hospital_admit_source'] == 'SDU', 'hospital_admit_source'] = 'ICU'
all_data.loc[all_data['hospital_admit_source'] == 'Chest Pain Center', 'hospital_admit_source'] = 'Other'
all_data.loc[all_data['hospital_admit_source'] == 'Observation', 'hospital_admit_source'] = 'Other'
all_data['hospital_admit_source'].fillna('Other', inplace = True)

all_data['icu_admit_source'].fillna(all_data['hospital_admit_source'], inplace = True)
all_data.loc[all_data['icu_admit_source'] == 'Operating Room', 'icu_admit_source'] = 'Operating Room / Recovery'
all_data.loc[all_data['icu_admit_source'] == 'Emergency Department', 'icu_admit_source'] = 'Accident & Emergency'
all_data.loc[all_data['icu_admit_source'] == 'Direct Admit', 'icu_admit_source'] = 'Other'

In [ ]:
categories = all_data.dtypes[all_data.dtypes == "object"].index
numeric_cols = all_data.dtypes[all_data.dtypes != "object"].index
# Imputing numerical values
all_data[numeric_cols] = all_data[numeric_cols].fillna(all_data[numeric_cols].mean())

In [ ]:
#dummy1 = pd.get_dummies(all_data[['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'train_test']])

all_dummies = pd.get_dummies(all_data[categories])

all_dummies.head()

In [ ]:
all_data = pd.concat([all_data, all_dummies], axis = 1)
all_data = all_data.drop(categories, axis = 1)
all_data.head()

# Outlier Detection

In [ ]:
train_data = all_data[all_data['train_data']==1]
train_data.shape

In [ ]:
test_data = all_data[all_data['train_data']==0]
test_data.shape

In [ ]:
def subset_by_iqr(df, column, whisker_width=1.5):
    """Remove outliers from a dataframe by column, including optional
       whiskers, removing rows for which the column value are
       less than Q1-1.5IQR or greater than Q3+1.5IQR.
    Args:
        df (`:obj:pd.DataFrame`): A pandas dataframe to subset
        column (str): Name of the column to calculate the subset from.
        whisker_width (float): Optional, loosen the IQR filter by a
                               factor of `whisker_width` * IQR.
    Returns:
        (`:obj:pd.DataFrame`): Filtered dataframe
    """
    # Calculate Q1, Q2 and IQR
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    # Apply filter with respect to IQR, including optional whiskers
    filter = (df[column] >= q1 - whisker_width*iqr) & (df[column] <= q3 + whisker_width*iqr)
    return df.loc[filter]


#imp_features = ['d1_glucose_max', 'd1_glucose_min', 'd1_creatinine_max', 'd1_bun_max', 'bmi', 'd1_hemaglobin_max', 'd1_sysbp_max', 'd1_wbc_max', 'd1_heartrate_max', 'pre_icu_los_days', 'apache_3j_diagnosis', 'd1_sysbp_min', 'd1_platelets_max']
#imp_features = ['d1_glucose_max', 'd1_glucose_min', 'd1_creatinine_max', 'd1_bun_max', 'bmi', 'd1_hemaglobin_max', 'd1_sysbp_max', 'd1_wbc_max', 'd1_heartrate_max', 'pre_icu_los_days', 'apache_3j_diagnosis', 'd1_sysbp_min', 'd1_platelets_max']

for feature in all_data.columns:
    cleaned_train_data = subset_by_iqr(train_data, feature, whisker_width=1.5)

cleaned_train_data.shape

In [ ]:
all_data=pd.concat([cleaned_train_data, test_data])
all_data.shape

# Removing correlated features

In [ ]:
def get_correlation(data, threshold):
    corr_col = set()
    corrmat = data.corr()
    for i in range(len(corrmat.columns)):
        for j in range(i):
            if abs(corrmat.iloc[i, j])> threshold:
                colname = corrmat.columns[i]
                corr_col.add(colname)
    return corr_col

In [ ]:
#all_data = all_data.drop('train_test')
corr_features = get_correlation(all_data, 0.80)
len(corr_features)

In [ ]:
all_data_uncorr = all_data.drop(labels=corr_features, axis = 1)
print('original size of data: ',all_data.shape)
print('After removing co related features: ',all_data_uncorr.shape)

In [ ]:
data = all_data_uncorr[all_data_uncorr.train_data==1].drop(['train_data'], axis =1)
x = data.drop(['diabetes_mellitus'], axis =1)
y = data['diabetes_mellitus']
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2, random_state = 40)
test = all_data_uncorr[all_data.train_data==0].drop(['train_data', 'diabetes_mellitus'], axis =1)

# Baseline Models

In [ ]:
cat_boost = CatBoostClassifier(verbose=0, n_estimators=100)
cat_boost.fit(x_train, y_train)
y_pred_cat_boost=cat_boost.predict(x_val)
# Performance Evaluation
print("Accuracy:",accuracy_score(y_val, y_pred_cat_boost))
print("Precision:",precision_score(y_val, y_pred_cat_boost))
print("Recall:",recall_score(y_val, y_pred_cat_boost))
print("F1 score:",f1_score(y_val, y_pred_cat_boost))

#print(classification_report(y_test, y_pred))
y_pred_proba = cat_boost.predict_proba(x_val)[:,1]
print("AUC score:",roc_auc_score(y_val, y_pred_proba))

plot_confusion_matrix(cat_boost, x_val, y_val)

In [ ]:
y_pred_cat_boost=cat_boost.predict(test)
submission = test_dataset[['encounter_id']]
submission['diabetes_mellitus'] = cat_boost.predict_proba(test)[:,1]
submission.to_csv('cat_boost_baseline.csv',index=False)
submission.head()

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, random_state=40, verbose = 1, n_jobs = -1)
random_forest.fit(x_train, y_train)
y_pred_rf=random_forest.predict(x_val)
# Performance Evaluation
print("Accuracy:",accuracy_score(y_val, y_pred_rf))
print("Precision:",precision_score(y_val, y_pred_rf))
print("Recall:",recall_score(y_val, y_pred_rf))
print("F1 score:",f1_score(y_val, y_pred_rf))

#print(classification_report(y_test, y_pred))
y_pred_proba = random_forest.predict_proba(x_val)[:,1]
print("AUC score:",roc_auc_score(y_val, y_pred_proba))

plot_confusion_matrix(random_forest, x_val, y_val)

In [ ]:
def plot_feature_importances(df):
    """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better.

    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance

    Returns:
        shows a plot of the 15 most importance features

        df (dataframe): feature importances sorted by importance (highest to lowest)
        with a column for normalized importance
        """

    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()

    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()

    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))),
            df['importance_normalized'].head(15),
            align = 'center', edgecolor = 'k')

    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()

    return df

In [ ]:
# Extract feature importances
features = list(x_train.columns)
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})
# Show the feature importances for the default features
feature_importances_sorted = plot_feature_importances(feature_importances)

In [ ]:
xgb = XGBClassifier(random_state =40)
xgb.fit(x_train, y_train)
y_pred_xgb=xgb.predict(x_val)
# Performance Evaluation
print("Accuracy:",accuracy_score(y_val, y_pred_xgb))
print("Precision:",precision_score(y_val, y_pred_xgb))
print("Recall:",recall_score(y_val, y_pred_xgb))
print("F1 score:",f1_score(y_val, y_pred_xgb))

#print(classification_report(y_test, y_pred))
y_pred_proba = xgb.predict_proba(x_val)[:,1]
print("AUC score:",roc_auc_score(y_val, y_pred_proba))

plot_confusion_matrix(xgb, x_val, y_val)

In [ ]:
#features = list(x_train.columns)
feature_importance_values = xgb.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})
# Show the feature importances for the default features
feature_importances_sorted = plot_feature_importances(feature_importances)

In [ ]:
y_pred_xgb=xgb.predict(test)
submission = test_dataset[['encounter_id']]
submission['diabetes_mellitus'] = xgb.predict_proba(test)[:,1]
submission.to_csv('xgb_baseline.csv',index=False)
submission.head()

In [ ]:
#lightGBM
lgb = lgb.LGBMClassifier(silent=False)
lgb.fit(x_train, y_train)
y_pred_lgb=lgb.predict(x_val)
# Performance Evaluation
print("Accuracy:",accuracy_score(y_val, y_pred_lgb))
print("Precision:",precision_score(y_val, y_pred_lgb))
print("Recall:",recall_score(y_val, y_pred_lgb))
print("F1 score:",f1_score(y_val, y_pred_lgb))

#print(classification_report(y_test, y_pred))
y_pred_proba = lgb.predict_proba(x_val)[:,1]
print("AUC score:",roc_auc_score(y_val, y_pred_proba))
plot_confusion_matrix(lgb, x_val, y_val)

In [ ]:
feature_importance_values = lgb.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})
# Show the feature importances for the default features
feature_importances_sorted = plot_feature_importances(feature_importances)

In [ ]:
y_pred_lgb=lgb.predict(test)
submission = test_dataset[['encounter_id']]
submission['diabetes_mellitus'] = lgb.predict_proba(test)[:,1]
submission.to_csv('lgb_baseline.csv',index=False)
submission.head()

In [ ]:
gbc=HistGradientBoostingClassifier(random_state=40).fit(x_train,y_train)
y_pred_gbc=gbc.predict(x_val)
# Performance Evaluation
print("Accuracy:",accuracy_score(y_val, y_pred_gbc))
print("Precision:",precision_score(y_val, y_pred_gbc))
print("Recall:",recall_score(y_val, y_pred_gbc))
print("F1 score:",f1_score(y_val, y_pred_gbc))

#print(classification_report(y_test, y_pred))
y_pred_proba = gbc.predict_proba(x_val)[:,1]
print("AUC score:",roc_auc_score(y_val, y_pred_proba))
plot_confusion_matrix(gbc, x_val, y_val)


In [ ]:
y_pred_gbc=gbc.predict(test)
submission = test_dataset[['encounter_id']]
submission['diabetes_mellitus'] = gbc.predict_proba(test)[:,1]
submission.to_csv('lgb_baseline.csv',index=False)
submission.head()

# **Hyperparameter Tuning**


**Tuning XGBoost Classifier**

In [ ]:
xgb = XGBClassifier(
                        n_estimators=200,
                        max_depth=8,
                        learning_rate=0.05,
                        subsample=0.8,
                        min_child_weight=10,
                        colsample_bytree=0.8
                       )
xgb.fit(x_train, y_train)
y_pred_xgb=xgb.predict(x_val)
# Performance Evaluation
print("Accuracy:",accuracy_score(y_val, y_pred_xgb))
print("Precision:",precision_score(y_val, y_pred_xgb))
print("Recall:",recall_score(y_val, y_pred_xgb))
print("F1 score:",f1_score(y_val, y_pred_xgb))

#print(classification_report(y_test, y_pred))
y_pred_proba = xgb.predict_proba(x_val)[:,1]
print("AUC score:",roc_auc_score(y_val, y_pred_proba))

plot_confusion_matrix(xgb, x_val, y_val)


In [ ]:
submission = test_dataset[['encounter_id']]
submission['diabetes_mellitus'] = xgb.predict_proba(test)[:,1]
submission.to_csv('xgb_tuned.csv',index=False)
submission.head()

# **LightGBM**

In [ ]:
#lightGBM
import lightgbm as lgb
lgb = lgb.LGBMClassifier(boosting_type= 'gbdt',
        objective= 'binary', metric= 'auc',
        learning_rate= 0.007, subsample= 1,
        colsample_bytree=  0.2,
        reg_alpha= 3, reg_lambda= 1,
        scale_pos_weight= 4, n_estimators= 10000,
        verbose= 1, max_depth=  -1, seed= 100,
        force_col_wise= True)
lgb.fit(x_train, y_train)
y_pred_lgb=lgb.predict(x_val)
# Performance Evaluation
print("Accuracy:",accuracy_score(y_val, y_pred_lgb))
print("Precision:",precision_score(y_val, y_pred_lgb))
print("Recall:",recall_score(y_val, y_pred_lgb))
print("F1 score:",f1_score(y_val, y_pred_lgb))

#print(classification_report(y_test, y_pred))
y_pred_proba = lgb.predict_proba(x_val)[:,1]
print("AUC score:",roc_auc_score(y_val, y_pred_proba))
plot_confusion_matrix(lgb, x_val, y_val)

In [ ]:
feature_importance_values = lgb.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})
# Show the feature importances for the default features
feature_importances_sorted = plot_feature_importances(feature_importances)

In [ ]:
y_pred_lgb=lgb.predict(test)
submission = test_dataset[['encounter_id']]
submission['diabetes_mellitus'] = lgb.predict_proba(test)[:,1]
submission.to_csv('lgb_tuned.csv',index=False)
submission.head()

In [ ]:
import lightgbm as lgb
xgb = XGBClassifier(
                        n_estimators=200,
                        max_depth=8,
                        learning_rate=0.05,
                        subsample=0.8,
                        min_child_weight=10,
                        colsample_bytree=0.8
                       )

#lightGBM
lgb = lgb.LGBMClassifier(boosting_type= 'gbdt',
        objective= 'binary', metric= 'auc',
        learning_rate= 0.007, subsample= 1,
        colsample_bytree=  0.2,
        reg_alpha= 3, reg_lambda= 1,
        scale_pos_weight= 4, n_estimators= 10000,
        verbose= 1, max_depth=  -1, seed= 100,
        force_col_wise= True)

gbc=HistGradientBoostingClassifier(random_state=40)
voting_classifier = VotingClassifier(estimators=[('xgb', xgb), ('lgb', lgb), ('gbc', gbc)], voting='soft', weights=[1, 3, 1])
voting_classifier.fit(x_train, y_train)
voting_pred = voting_classifier.predict(x_val)

# Performance Evaluation
print("Accuracy:",accuracy_score(y_val, voting_pred))
print("Precision:",precision_score(y_val, voting_pred))
print("Recall:",recall_score(y_val, voting_pred))
print("F1 score:",f1_score(y_val, voting_pred))

#print(classification_report(y_test, y_pred))
y_pred_proba = voting_classifier.predict_proba(x_val)[:,1]
print("AUC score:",roc_auc_score(y_val, y_pred_proba))
plot_confusion_matrix(voting_classifier, x_val, y_val)

In [ ]:
submission = test_dataset[['encounter_id']]
submission['diabetes_mellitus'] = voting_classifier.predict_proba(test)[:,1]
submission.to_csv('voting_baseline.csv',index=False)
submission.head()

References

* https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html#sphx-glr-auto-examples-compose-plot-digits-pipe-py

* https://www.kaggle.com/iamleonie/wids-datathon-2021-diabetes-detection

* https://www.kaggle.com/paolagasp/wids-eda-building-models

* https://towardsdatascience.com/improving-random-forest-in-python-part-1-893916666cd

* https://www.kaggle.com/cosmosankur/xgboost-classifier#SELECTING-TRAIN-AND-TEST-DATA

* https://www.kaggle.com/parulpandey/starter-code-with-baseline

* https://www.datacamp.com/community/tutorials/categorical-data

* https://www.kaggle.com/srinidhi123/wids2020-ensemblelearning-votingclassifier/comments

* https://www.kaggle.com/willkoehrsen/automated-model-tuning

* https://www.kaggle.com/c/widsdatathon2021/discussion/209053

* https://www.kaggle.com/nasere/wids2021-using-lightgbm-encoding